## Подключение к PT NAD

ptnad-client позволяет выполнять запросы к системе с использованием различных способов аутентификации.

### Локальное подключение

Для быстрого тестирования и отладки можно использовать локальное подключение. В качестве дефолтных значений используется логин "administrator" и пароль "Administr@t0r".

In [ ]:
from ptnad import PTNADClient

# Подключение к локальному инстансу PT NAD (пример для тестовой среды)
client = PTNADClient("https://1.3.3.7", verify_ssl=False)

# Установка логина и пароля
client.set_auth(username="administrator", password="Administr@t0r")

# Подключаемся к сервису
client.login()


### Подключение через SSO

Для подключения через SSO необходимо использовать auth_type="sso" и параметры, которые можно получить из файла /opt/ptsecurity/etc/iam_cookie.json на сервере PT NAD.

Пример структуры, получаемой при выводе содержимого "iam_cookie.json":

```bash
cat /opt/ptsecurity/etc/iam_cookie.json
```

```json
{
    "iam": {
        "id": "ptnad",
        "name": "PT NAD",
        "redirect_url": "https://1.3.3.7",
        "secret": "11111111-abcd-asdf-12334-0123456789ab",
        "token": "your_token",
        "token_type": "reference",
        "url": "https://siem.example.local:3334",
        "verify": false
    },
    "tms": {
        "app_id": "your_app_id",
        "display_name": "PT NAD",
        "endpoint": "https://1.3.3.7",
        "request_id": "your_request_id",
        "token": "your_token",
        "url": "https://siem.example.local:3334"
    }
}
```

Для подключения, необходимо взять информацию из ключей "id", "secret" и "url".
В качестве auth_type указываем "sso", добавляем свои логин/пароль и оставшиеся поля из iam_cookie.json, где
- client_id = id
- client_secret = secret
- sso_url = url

По итогу получаем:

```python
client = PTNADClient("https://1.3.3.7", verify_ssl=False)
client.set_auth(
    auth_type="sso",
    username="user",
    password="pass",
    client_id="ptnad",
    client_secret="11111111-abcd-asdf-12334-0123456789ab",
    sso_url="https://siem.example.local:3334"
)
client.login()
```

Ссылка на код, где можно подробнее изучить механизмы авторизации: [Code Reference](https://security-experts-community.github.io/ptnad-client/reference)

### Примеры работы с библиотекой

#### BQL-запросы

В этом примере демонстрируется работа с кастомными фильтрами, хранящимися в файле "docs\data\filters.json", и выполнение BQL-запросов к PT NAD. Для примера выберем только два поля: IP-адрес источника (src.ip) и IP-адрес назначения (dst.ip):
- Локально подключаемся PT NAD.
- Загружаем фильтры из файла filters.json, где каждый фильтр содержит BQL-фрагмент, имя и категорию (есть возможность выбора языка).
- Формируем базовый запрос с временным диапазоном (последние 3 дня).
- Объединяем базовый запрос с фильтрами.
- Группируем запросы по категориям.
- Выполняем каждый запрос и выводим результаты с указанием:
    - категории фильтра;
    - его названия;
    - полного BQL-запроса;
    - результата.

Более подробно про возможные параметры фильтрации и их значения можно почитать тут - [PT NAD Справка](https://help.ptsecurity.com/ru-RU/projects/nad/12.2/help/7083891595)

In [ ]:
import json
from datetime import datetime, timedelta
from ptnad import PTNADClient

client = PTNADClient("https://1.3.3.7", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

with open("../data/filters.json", "r", encoding="utf-8") as f:
    data = json.load(f)

now = datetime.now()
start_time = (now - timedelta(days=3)).strftime("%Y-%m-%dT%H:%M:%S")
end_time = now.strftime("%Y-%m-%dT%H:%M:%S")

base_query = f"SELECT src.ip, dst.ip FROM flow WHERE start > '{start_time}' AND end < '{end_time}'"

grouped_queries = {}

for item in data["filters"]:
    category = item["category"]["en"]
    name = item["name"]["en"]
    filter_query = item["filter"].strip()
    compiled_filter = client.filters.compile(filter_query)

    if filter_query:
        full_query = f"{base_query} AND {compiled_filter}"
        if category not in grouped_queries:
            grouped_queries[category] = []
        grouped_queries[category].append((name, full_query))

for category, filters in grouped_queries.items():
    print(f"\n▶️ Категория: {category}")
    for name, query in filters:
        result = client.bql.execute(query)
        print(f"\n🔹 {name}\n{query}")
        print("Results:")
        if isinstance(result, list) and result:
            for i, row in enumerate(result, 1):
                print(f"{i}. {row}")
        else:
            print("Нет результатов.")

#### Monitoring

Получение общего статуса системы:

In [ ]:
from ptnad import PTNADClient
from ptnad.api.monitoring import MonitoringAPI

client = PTNADClient("https://1.3.3.7", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

monitoring = MonitoringAPI(client)

status = monitoring.get_status()

print(f"Статус: {status.status}")
for problem in status.problems:
    print(f"{problem}")

Получение списка всех триггеров:

In [ ]:
from ptnad import PTNADClient
from ptnad.api.monitoring import MonitoringAPI

client = PTNADClient("https://1.3.3.7", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

monitoring = MonitoringAPI(client)

triggers = monitoring.get_triggers()

for trig in triggers:
    print(f"\nTrigger ID: {trig.id}")
    print(f"Тип: {trig.type}")
    print(f"Статус: {trig.status}")
    print(f"Последнее обновление: {trig.updated}")

#### Replists

Получение всех списков:

In [ ]:
from ptnad import PTNADClient
from ptnad.api.replists import RepListsAPI

client = PTNADClient("https://1.3.3.7", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

replists = RepListsAPI(client)

lists = replists.get_all_lists()
print(f"Общее количество: {len(lists)}")

for r in lists:
    print(f"- [{r['id']}] {r['name']} ({r['type']}, items: {r['items_count']})")


Поиск по ключевому слову:

In [ ]:
from ptnad import PTNADClient
from ptnad.api.replists import RepListsAPI

client = PTNADClient("https://1.3.3.7", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

replists = RepListsAPI(client)

results = replists.get_lists(search="APT")

for r in results:
    print(f"{r['name']}")

Создание нового списка:

In [ ]:
from ptnad import PTNADClient
from ptnad.api.replists import RepListsAPI

client = PTNADClient("https://1.3.3.7", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

replists = RepListsAPI(client)

new_list = replists.create_list(
    name="test_list_01",
    type="ip",
    color="1",
    description="Тестовый список IP-адресов",
    content="1.1.1.1\n2.2.2.2"
)
print(f"Список создан: {new_list['id']}")

Обновление списка:

In [ ]:
from ptnad import PTNADClient
from ptnad.api.replists import RepListsAPI

client = PTNADClient("https://1.3.3.7", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

replists = RepListsAPI(client)

updated = replists.update_list(list_id=new_list["id"], description="Обновлённый тестовый список IP-адресов")
print("Описание обновлено:", updated["description"])

Удаление списка:

In [ ]:
from ptnad import PTNADClient
from ptnad.api.replists import RepListsAPI

client = PTNADClient("https://1.3.3.7", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

replists = RepListsAPI(client)

replists.delete_list(list_id=new_list["id"])
print("Список удалён")

#### Signatures

Получение всех классов сигнатур:

In [ ]:
from ptnad import PTNADClient
from ptnad.api.signatures import SignaturesAPI

client = PTNADClient("https://1.3.3.7", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

signatures = SignaturesAPI(client)

classes = signatures.get_classes()

for cls in classes:
    print(f"{cls['title']} (priority: {cls['priority']})")


Получение полной информации по одной сигнатуре:

In [ ]:
from ptnad import PTNADClient
from ptnad.api.signatures import SignaturesAPI

client = PTNADClient("https://1.3.3.7", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

signatures = SignaturesAPI(client)

rules = signatures.get_rules(search="ftp", cls="Network scan")

for r in rules:
    print(f"\nRule #{r['sid']} - {r['msg']}")
    print(f"Vendor: {r['vendor']}, Enabled: {r['enabled']}")